In [ ]:
import numpy as np
import pandas as pd

import pennylane as qml
from pennylane.templates import AngleEmbedding

from sklearn.preprocessing import MinMaxScaler

In [ ]:
filename = 'curated-solubility-dataset.csv'
df = pd.read_csv(filename, sep=',')

# select the 300 samples used in the usage example
df_subset = df[(df['Group'] == 'G5')][:300]

# get the first four features from the dataframe
X = df_subset[['MolWt', 'MolLogP', 'MolMR', 'HeavyAtomCount']].to_numpy()
num_feature = 4

# scale the inputs to range (0, 1) for the input roatational angles in quantum circuits
scaler = MinMaxScaler().fit(X)
X_scaled = scaler.transform(X)

In [ ]:
# prepare the quantum simulator
n_qubits = num_feature
dev_kernel = qml.device("lightning.qubit", wires=n_qubits)

# only consider the expectation value of measuring all 0s
# ignore other possible quantum states
projector = np.zeros((2 ** n_qubits, 2 ** n_qubits))
projector[0, 0] = 1

# define the quantum circuits to construct the kernel matrix
@qml.qnode(dev_kernel)
def feature_mapping(x1):
    AngleEmbedding(x1, wires=range(n_qubits), rotation='X')


In [ ]:
print(qml.draw(feature_mapping, level="device")(X_scaled[0]))

0: ──RX(0.70)─┤  
1: ──RX(0.80)─┤  
2: ──RX(0.65)─┤  
3: ──RX(0.68)─┤  
